Given:
- sparseframe pixels file (s, f, omega, intensity)
- parameters file
- spatial distortion images (sraw -> sc, fraw -> fc)
- ubi file for list of grains
    
ForEach pixel:
- assign it to one or more grains with h,k,l,sign(eta) label
- best grain + hklid
- second best grain + hklid -> to determine whether this is an overlap problem / twin

In [ ]:
import sys, time
start = time.time()
import numpy as np, pylab as pl
from ImageD11 import transform, parameters, cImageD11, indexing, columnfile, sym_u, blobcorrector, grain
import fabio, h5py


PLOT = False
%matplotlib inline

In [ ]:
# put in transform or blobcorrector ?

class Lut( object ):
                 
    pnames = ( "y_center", "z_center", "y_size", "z_size",
               "distance", "wavelength",
               "tilt_x","tilt_y","tilt_z",
               "o11", "o12", "o21", "o22",
               "wedge", "chi", "dxfile", "dyfile", "spline", "shape" )
                 
    def __init__( self, pars ):
        
        self.pars = {}
        for p in self.pnames:
            if p in pars:
                self.pars[p] = pars[p] # make a copy
        if 'dxfile' in pars:
            # slow/fast coordinates on image at pixel centers
            self.df = fabio.open( pars['dxfile'] ).data
            self.ds = fabio.open( pars['dyfile'] ).data
            self.shape = s = self.ds.shape
            self.pars['shape'] = s
            slow, fast = np.mgrid[ 0:s[0], 0:s[1] ]
            self.sc = slow + self.ds
            self.fc = fast + self.df
        elif 'spline' in pars: # need to test this...
            b = blobcorrector.correctorclass( self.pars['spline'] )
            s = self.pars['shape']
            self.fc, self.sc = b.make_pixel_lut( s ) 
            slow, fast = np.mgrid[ 0:s[0], 0:s[1] ]
            self.df = self.fc - fast
            self.ds = self.sc - slow
        # scattering angles:
        self.tth, self.eta = transform.compute_tth_eta( 
            (self.sc.ravel(), self.fc.ravel()), **self.pars )
        # scattering vectors:
        self.k = transform.compute_k_vectors( self.tth, self.eta, 
                                              self.pars.get('wavelength') )
        self.tth.shape = s
        self.eta.shape = s
        self.k.shape = (3, s[0], s[1])
    
    def spatial(self, sraw, fraw):
        """ applies a spatial distortion to sraw, fraw (for peak centroids) """
        si = np.round(sraw.astype(int)).clip( 0, self.shape[1] - 1 )
        fi = np.round(fraw.astype(int)).clip( 0, self.shape[1] - 1 )
        sc = sraw + self.ds[ si, fi ]
        fc = fraw + self.df[ si, fi ]
        return sc, fc     
        
    def __repr__(self):
        """ print yourself in a way we can use for eval """
        sp = "\n".join( [ "%s : %s,"%(repr(p), repr(self.pars[p])) for p in self.pnames
                         if p in self.pars ] )
        return "Lut( { %s } )"%(sp)

In [ ]:
%%time 
pars = parameters.read_par_file( "../test/pixelmapper/CeO2.par" ).parameters
pars['dxfile'] = "/data/id11/nanoscope/Eiger/spatial_20210415_JW/e2dx.edf"
pars['dyfile'] = "/data/id11/nanoscope/Eiger/spatial_20210415_JW/e2dy.edf"
tabl = Lut( pars )
print(tabl)

In [ ]:
si, fi = int( tabl.pars['z_center'] ), int( tabl.pars['y_center'])
ref = tabl.k[:,si,fi]
sub_pixel_factor = 0
nfac = 0
for i in range(-1,2):
    for j in range(-1,2):
        dk = (tabl.k[:,si+i,fi+j]-ref)
        if i!=0 or j!=0:
            # actual distance will be from the center to 1/4 and 3/4 points
            sf = np.dot(dk,dk)/(i*i+j*j)/16
            sub_pixel_factor += sf
            nfac += 1

sub_pixel_factor /= 8
print(sub_pixel_factor)   # FIXME : do the integral!

In [ ]:
def MB(num):
    return "%.3f MB"%(num / (1024*1024) )

with h5py.File("../test/pixelmapper/silicon_fullscan_sparse.h5" ,"r") as hin:
    for scan in hin:
        s = hin[scan]
        print(scan,list(s))
        print(dict(s.attrs))
        for column in list(s['measurement']):
            print(column, s['measurement'][column].shape)
        nnz = s['nnz'][:]
        ipt = np.cumsum(nnz)
        print("pixels",ipt[-1],"per frame avg:", ipt[-1]/len(nnz))
        nbytes = 0
        for name in 'row','col','frame','intensity','measurement/rot_center':
            a = s[name]
            b = a.dtype.itemsize * a.size
            print(name, a.shape, a.dtype, a.size, MB(b))
            nbytes += b
        print(MB(nbytes))

In [ ]:
def tocolf(pks):
    titles = list(pks.keys())
    colf = columnfile.newcolumnfile( titles=titles )
    nrows = len(pks[titles[0]])
    colf.nrows = nrows
    colf.set_bigarray( [ pks[t] for t in titles ] )
    return colf

class SparseScan( object ):
    
    omeganames = ['measurement/rot_center', 'measurement/rot',
                  'measurement/diffrz_center', 'measurement/diffrz']
    dtynames   = ['measurement/dty_center', 'measurement/dty',
                  'measurement/diffty_center', 'measurement/diffty']
    
    def __init__( self, hname, scan ):
        """
        hname : file coming from a sparse segmentation
        scan : a scan within that file
        motors : which motor channels to (try) to read
        
        assumes the scan fits into memory (could be problematic)
        """
        with h5py.File(hname,"r") as hin:
            grp = hin[scan]
            self.shape = ( int(v) for v in ( grp.attrs['nframes'], 
                                            grp.attrs['shape0'], 
                                            grp.attrs['shape1'] ) )
            self.motors = {}
            for name, motors in [ ('omega',self.omeganames),
                                  ('dty',self.dtynames) ]:
                for motor in motors:
                    if motor in grp:
                        self.motors[ name ] = grp[motor][:]
                        break
                
            self.nnz = grp['nnz'][:]
            self.ipt = np.concatenate( ( (0,) , np.cumsum(self.nnz, dtype=int) ) )
            self.frame  = grp['frame'][:]
            self.row = grp['row'][:]
            self.col = grp['col'][:]
            self.intensity = grp['intensity'][:]
            
    def cplabel(self, threshold = 0 ):
        """ Label pixels using the connectedpixels assigment code
        Fills in:
           self.nlabels = number of peaks per frame
           self.labels  = peak labels (should be unique)
           self.total_labels = total number of peaks
        """
        self.nlabels = np.zeros( len(self.nnz), np.int32 )
        self.labels = np.zeros( len(self.row), "i")
        nl = 0
        for i, npx in enumerate( self.nnz ):
            s = self.ipt[i]
            e = self.ipt[i+1]
            if npx > 0:
                self.nlabels[i] = cImageD11.sparse_connectedpixels(
                    self.intensity[ s : e ],
                    self.row[ s : e ],
                    self.col[ s : e ],
                    threshold,
                    self.labels[ s : e ] )
                assert (self.labels[ s : e ] > 0).all()
                self.labels[ s : e ] += nl
            else:
                self.nlabels[i] = 0
            nl += self.nlabels[i]
        self.total_labels = nl

                
    def lmlabel(self, threshold = 0 ):
        """ Label pixels using the localmax assigment code
        Fills in:
           self.nlabels = number of peaks per frame
           self.labels  = peak labels (should be unique)
           self.total_labels = total number of peaks
        """
        self.nlabels = np.zeros( len(self.nnz), np.int32 )
        self.labels = np.zeros( len(self.row), "i")
        # temporary workspaces
        npxmax = self.nnz.max()
        vmx = np.zeros( npxmax, np.float32 )
        imx = np.zeros( npxmax, 'i' )
        nl = 0
        for i, npx in enumerate( self.nnz ):
            s = self.ipt[i]
            e = self.ipt[i+1]
            if npx > 0:
                self.nlabels[i] = cImageD11.sparse_localmaxlabel(
                    self.intensity[ s : e ],
                    self.row[ s : e ],
                    self.col[ s : e ],
                    vmx[:npx],
                    imx[:npx],
                    self.labels[s : e] )
                assert (self.labels[s:e] > 0).all()
                self.labels[ s : e ] += nl
            else:
                self.nlabels[i] = 0
            nl += self.nlabels[i]
        self.total_labels = nl

            
    def moments(self):
        """ Computes the center of mass in s/f/omega 
        returns a columnfile
        """
        pks = {}
        for name , weights in [ ('Number_of_pixels', None),
                                ('s_raw', self.row ),
                                ('f_raw', self.col ),
                                ('omega', self.motors['omega'][self.frame]) ]:
            pks[name] = np.bincount( self.labels, weights,
                                     minlength = self.total_labels+1 )[1:]
            if weights is not None:
                pks[name] /= pks['Number_of_pixels']
        return tocolf(pks)


In [ ]:
s = SparseScan( "../test/pixelmapper/silicon_fullscan_sparse.h5", "1.1" )

In [ ]:
s.motors

In [ ]:
%%time
s.cplabel()
if PLOT:
    pl.figure()
    pl.plot(s.nlabels)
print(s.total_labels)

In [ ]:
%time
c = s.moments()

In [ ]:
sc, fc = tabl.spatial( c.s_raw, c.f_raw )
c.addcolumn( sc, 'sc')
c.addcolumn( fc, 'fc')
c.parameters.loadparameters("../test/pixelmapper/CeO2.par")
c.updateGeometry()

In [ ]:
if PLOT:
    pl.figure()
    pl.plot(c.tth, c.Number_of_pixels * c.tth,",")

In [ ]:
mask = (c.tth * c.Number_of_pixels) > 1000
c.filter(mask)

In [ ]:
a0 = 5.43094
[c.parameters.set("cell__%s"%(abc), a0) for abc in 'abc']
i = indexing.indexer_from_colfile( c )
i.assigntorings()

In [ ]:
i.minpks = 10000
i.hkl_tol = 0.1
i.ring_1 = i.ring_2 = 21
i.find()
i.scorethem()
print("Got",len(i.ubis),"grains")
i.histogram_drlv_fit()
if PLOT:
    pl.figure()
    for j in range(len(i.ubis)):
        pl.plot(i.bins[1:], i.histogram[j],"-")

In [ ]:
cubic = sym_u.cubic()
ubis = i.ubis = [ sym_u.find_uniq_u( ubi, cubic ) for ubi in i.ubis]
ubis[0]

In [ ]:
indexing.ubitocellpars(ubis[0])

In [ ]:
tabl.k.shape, len(s.row)

In [ ]:
%%time
pomega = s.motors['omega'][s.frame]
gve = transform.compute_g_from_k( tabl.k[:, s.row, s.col] , 
                                  s.motors['omega'][s.frame] )
gve.shape

In [ ]:
%time 
ga = np.empty( s.col.shape )
ga.fill(-1)
ge = np.ones( s.col.shape )
for k, ubi in enumerate( i.ubis ):
    hkl = np.dot( ubi, gve )
    print(hkl.shape)
    gcalc = np.dot( np.linalg.inv(ubi), np.round(hkl) )
    gerr = gcalc - gve
    modge = (gerr**2).sum(axis=0)
    best = (modge < ge)
    ga[best] = k

In [ ]:
ptth = tabl.tth[ s.row, s.col ]
peta = tabl.eta[ s.row, s.col ]
if PLOT:
    pl.figure()
    pl.plot( ptth, modge, "," )
    pl.semilogy()

In [ ]:
m = modge<0.001
if PLOT:
    pl.figure()
    pl.plot( ptth[  m ], peta[ m], ",")
    pl.plot( ptth[ ~m ], peta[~m], ",")
    pl.figure()
    pl.plot( pomega[m], s.intensity[m], ',')
    pl.plot( pomega[~m], s.intensity[~m], ',')
    pl.semilogy()

Here, we can now have all the pixels assigned to their grain and hkls 

... to do next:
- build a refinement engine using intensity weighting
- deal with saturated peaks

In [ ]:
print("Total time",time.time()-start)

In [ ]:
ubi = i.ubis[0]
print(gve.shape)
h,k,l = np.round( np.dot( ubi, gve ) ).astype(int)
gcalc = np.dot( np.linalg.inv(ubi), (h,k,l) )
mgerr  = ((gve - gcalc)**2).sum(axis=0)
m = mgerr < 0.001
se = np.sign(peta).astype(int)

In [ ]:
%%time
sortkeys = ( h,k,l,se )
order = np.lexsort( sortkeys )
print(order.shape)

In [ ]:
%%time

# indexing from 0 -> h.max() ...  -h.min()->end
nh = h.max() - h.min() + 1
nk = k.max() - k.min() + 1
nl = l.max() - l.min() + 1
ne = 3
gvar = np.zeros( (3, 3, nh, nk, nl, ne), float )
gavg = np.zeros( (3, nh, nk, nl, ne), float )
f = np.zeros( (nh, nk, nl, ne), float )
e = np.zeros( (9,) ,dtype=np.float).reshape(3,3)
# this does not accumulate !
# f[h[m],k[m],l[m],se[m]] += s.intensity[m]
# gavg[:, h[m], k[m], l[m], se[m]] += gve[:,m]*s.intensity[m]
import numba
@numba.njit
def incr( m, h, k, l, se, gve, sig, gavg, f, gvar, e  ):
    # Compute the mean g-vectors in mask:
    for p in range(len(m)):
        if m[p]:
            gavg[:, h[p],k[p],l[p],se[p]] += sig[p] * gve[:,p]
            f[ h[p],k[p],l[p],se[p]] += sig[p]
    # Get the mean
    for i in range(f.size):
        v = f.flat[i] 
        if v > 0:
            gavg[0].flat[i] /= v
            gavg[1].flat[i] /= v
            gavg[2].flat[i] /= v
    # Now get the variances:
    #       subpixel_factor = 1.7e-6     # From above. This is the in-plane error
    #                                    # not rotational, which depends on omega step.
    #                 FIXME - not entirely correct yet. Problem goes back a long time.
    for p in range(len(m)):
        if m[p]:
            dg = gve[:,p] - gavg[:,h[p],k[p],l[p],se[p]]
            # this loop is np.outer( dg, dg )
            for i in range(3):
                for j in range(3):
                    gvar[i, j, h[p],k[p],l[p],se[p]]  += (dg[i]*dg[j])*sig[p]
            for i in range(3):
                gvar[ i, i, h[p], k[p], l[p], se[p]]  += sub_pixel_factor*sig[p]
            # We are missing a pixel size contribution here. The pixel 
            # is not a point in space. This wants to sum up over, for 
            # example 2 subpixels as (sig/2 times:)
            #    outer( dg + p, dg + p ) + ...->    dg(x)dg + 2 dg(x)p + p(x)p
            #    outer( dg - p, dg - p ) + ...->    dg(x)dg - 2 dx(x)p + p(x)p
                    
    for p in range(f.size):
        v = f.flat[p]
        if v > 0:
            for i in range(3):
                for j in range(3):
                    e[i,j] = gvar[i,j].flat[p] / v
            d = np.linalg.det(e)
            if d <= 0:
                print('error')
                print(e, p, v, d)
                return None
            ie = np.linalg.inv(e)
            if ie[0,0] < 0:
                print('error -ve')
                print( e, p, v, d )
                print(ie)
                print( 'y',gvar[:,:].flat[p] )
                return None
            for i in range(3):
                for j in range(3):
                     gvar[i,j].flat[p] = ie[i,j]
            

incr( m, h, k, l, se, gve, s.intensity, gavg, f, gvar, e )

In [ ]:
%%time
gvar = np.zeros( (3, 3, nh, nk, nl, ne), float )
gavg = np.zeros( (3, nh, nk, nl, ne), float )
f = np.zeros( (nh, nk, nl, ne), float )
incr( m, h, k, l, se, gve, s.intensity, gavg, f, gvar, e )

In [ ]:
# option 1: take the average of the observed pixels
np.dot(ubi, gavg[:,1,1,1,1]),np.dot(ubi, gavg[:,3,1,1,-1]),
# option 2: take the computed gcalc position
gcalc = np.dot( np.linalg.inv( ubi ), np.round( np.dot( ubi, gve ) ) )
gerr = gcalc - gve

In [ ]:
# check
gvar[:,:,3,1,1,1], gavg[:,3,1,1,1], f[3,1,1,1]

In [ ]:
# example 
dg = np.dot( np.linalg.inv(ubi), (3,1,1) ) - gavg[:,3,1,1,1]
np.dot(dg, np.dot( gvar[:,:,3,1,1,1], dg ))

In [ ]:
# example
dg = np.dot( np.linalg.inv(ubi), (5,3,3) ) - gavg[:,5,3,3,1]
print(dg,'\n', gvar[:,:,5,3,3,1], f[5,3,3,1])
np.dot(dg, np.dot( gvar[:,:,5,3,3,1], dg ))

In [ ]:
%%time
gouter = np.einsum( 'ik,jk->ijk', gerr, gerr )
assert gouter.shape == (3,3,gerr.shape[1])
assert np.allclose( np.outer( gerr[:,3], gerr[:,3] ), gouter[:,:,3] )

In [ ]:
# MAM'x = MAd
#  ... A  == gvar               3x3
#  ... d  == gobs - gcalc       3
#  ... M' == dg/dvariable       9x3
#  ... x  == variables          9
#
#  9x3.3x3.3x9.9 = 9x3.3x3.3


M = np.zeros( (9,3), float )
LSM = np.zeros( (9,9), float )
RHS = np.zeros( (9,), float)

def dg_dUB( M, h, k, l ):
    """ 
    dg_dUB :
      gx = UB_00.h UB_01.h UB_02.l
      gy = UB_10.h UB_11.h UB_12.l
      gz = UB_20.h UB_21.h UB_22.l
    M is 9x3 :
      UB, g
    """
    #          x y z
    M[:,:] = ((h,0,0),
              (k,0,0),
              (l,0,0),
              (0,h,0),
              (0,k,0),
              (0,l,0),
              (0,0,h),
              (0,0,k),
              (0,0,l) )
    return M
        
ub = np.zeros((3,3), float) # Linear problem. hkl already assigned
ub = np.linalg.inv( ubis[0]) # not for error estimation - need chi^2
npk = 0
g_to_fit = []
h_to_fit = []
XI2 = []
fsum = f.sum()
for ih in  range(h.min(), h.max()+1):
    for ik in  range(k.min(), k.max()+1):
        for il in  range(l.min(), l.max()+1):
            for sign_eta in (-1,1):
                # total intensity for this peak
                signal = f[ih,ik,il,sign_eta]/fsum
                if signal == 0:
                    continue
                npk += 1
                # This matrix already carries an intensity weighting factor
                A = gvar[:,:,ih,ik,il,sign_eta]   # 3x3
                # A = np.eye(3)
                gobs = gavg[:,ih,ik,il,sign_eta]  # 3
                gcalc = np.dot( ub, (ih,ik,il) )  # 3
                ge = gobs - gcalc
                # Contribution to fit
                scor = np.dot(ge, np.dot( A, ge ) ) 
                assert scor >= 0
                cut = 2 * 10
                weight = cut*cut/(cut*cut+scor*scor) # weight goes down as scor goes up
                XI2.append( scor*weight )
                # For later debugging
                if 1:
                    g_to_fit.append( gobs )
                    h_to_fit.append( (ih,ik,il) )
                M   = dg_dUB( M, ih, ik, il )
                MA   = np.dot( M , A )
                LSM += np.dot( MA,  M.T ) * weight
                RHS += np.dot( MA, ge )   * weight
                # import pdb;pdb.set_trace()
#LSM *= npk/XI2
#RHS *= npk/XI2
iMAT = np.linalg.inv( LSM )
ecorfac = np.sum( XI2 ) / (npk - len(RHS) )   # potentially a factor of 2 here in scipy?
pCOV = iMAT * ecorfac
shifts = np.dot( iMAT, RHS) + ub.ravel()
print(npk)
print(shifts)
print(np.sqrt(np.diag(pCOV)))
pl.figure()
pl.hist( XI2, bins=128 )
pl.ylabel("Number of peaks")
pl.xlabel("chi^2 contribution")
pl.figure()
pl.imshow(pCOV)
pl.colorbar()

In [ ]:
ubfit = shifts.reshape(3,3)
ubi_fit = np.linalg.inv( ubfit )
print(ubi_fit)
print(indexing.ubitocellpars(ubi_fit))
afit = pow(np.linalg.det(ubi_fit),1/3)
print(grain.grain( ubi_fit ).eps_grain( [afit,afit,afit,90,90,90] ))

In [ ]:
g_to_fit = np.array(g_to_fit, float)
h_to_fit = np.array(h_to_fit, float)
sum_h_outer_h = np.einsum( 'ki,kj->ij', h_to_fit, h_to_fit )
sum_h_outer_g = np.einsum( 'ki,kj->ij', h_to_fit, g_to_fit )
ubi_fit2 = np.dot( np.linalg.inv( sum_h_outer_g ), sum_h_outer_h ).T
print(ubi_fit2)
print(indexing.ubitocellpars(ubi_fit2))
afit = pow(np.linalg.det( ubi_fit2 ),1/3)
print(grain.grain( ubi_fit2 ).eps_grain( [afit,afit,afit,90,90,90] ))

In [ ]:
i.ubis[0]

In [ ]:
ubi_fit3 = indexing.refine( ubi_fit, g_to_fit, 0.4 )
print(ubi_fit3)
print(indexing.ubitocellpars(ubi_fit3))